## Cash-Karp Runge-Kutta method with adaptive stepwise control

Evolve the system of equations: dy/dx = z, dz/dx = -y

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### Define our coupled derivatives to integrate

In [ ]:
def dydx(x, y):
    #set the derivatives
    #dydx = z
    #dzdx = -y
    
    #set y = y[0] and z = y[1]
    
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    return y_derivs

### Define the 4th order RK method

In [ ]:
def rk4_mv_core(dydx, xi, yi, nv, h):
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    #define x at 1/2 step
    x_ipoh = xi + 0.5*h
    
    #define x at 1 step
    x_ipo = xi + h
    
    #declare a temporary y array
    y_temp = np.zeros(nv)
    
    #get k1 valyes
    y_derivs = dydx(xi, yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh, y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh, y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipo, y_temp)
    k4[:] = h*y_derivs[:]
    
    #advance y by a step h
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    
    return yipo

### Adaptive step-size control

In [ ]:
def rk6_mv_core(dydx, xi, yi, nv, h):
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    #define coefficients
    c = [0,0,1/5,3/10,3/5,1,7/8]
    a1 = [0,1/5,3/40,3/10,-11/54,1631/55296]
    a2 = [0,0,9/40,-9/10,5/2,175/512]
    a3 = [0,0,0,6/5,-70/27,575/13824]
    a4 = [0,0,0,0,35/27,44275/110592]
    a5 = [0,0,0,0,0,253/4096]
    bi = [37/378,0,250/621,125/594,0,512/1771]
    bI = [2825/27648,0,18575/48384,13525/55296,277/14336,1/4]
    
    k1 = h * dydx(xi, yi)
    k2 = h * dydx(xi + c[2]*h, yi + a1[1]*k1)
    k3 = h * dydx(xi + c[3]*h, yi + a1[2]*k1 + a2[2]*k2)
    k4 = h * dydx(xi + c[4]*h, yi + a1[3]*k1 + a2[3]*k2 + a3[3]*k3)
    k5 = h * dydx(xi + c[5]*h, yi + a1[4]*k1 + a2[4]*k2 + a3[4]*k3 + a4[4]*k4)
    k6 = h * dydx(xi + c[6]*h, yi + a1[5]*k1 + a2[5]*k2 + a3[5]*k3 + a4[5]*k4 + a5[5]*k5)
    
    yipo = (bi[0] - bI[0])*k1 + (bi[1] - bI[1])*k2 + (bi[2] - bI[2])*k3 + (bi[3] - bI[3])*k4 + (bi[4] - bI[4])*k5 + (bi[5] - bI[5])*k6
    
    return yipo

### Define an adaptive step size driver for RK4

In [ ]:
def rk4_mv_ad(dydx, x_i, y_i, nv, h, tol):
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of itterations
    imax = 1000
    
    #set an iteration variabel
    i = 0
    
    #create an error
    Delta = np.full(nv, 2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/ tol > 1.0):
        #estimate our error by taking one step of size h vs. two steps of size h/2
        y_2 = rk6_mv_core(dydx, x_i, y_i, nv, h_step)
        y_1 = rk6_mv_core(dydx, x_i, y_i, nv, 0.5*h_step)
        y_11 = rk6_mv_core(dydx, x_i + 0.5*h_step, y_1, nv, 0.5*h_step)
        
        #compute an error
        Delta = np.fabs(y_2 - y_11)
        
        #if the error is too large, take a smaller step
        if(Delta.max()/tol > 1.0):
            #our error is too large, decreases the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(i >= imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate
        i += 1
        
    #next time try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step

### Define a wrapper for RK4

In [ ]:
def rk4_mv(dfdx, a, b, y_a, tol):
    #dfdx is the derivate with respect to x
    #a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small!
    h = 1.0e-4 * (b - a)
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration varible
    i = 0
    
    #set the number of coupled odes to the size of y_a
    nv = len(y_a)
    
    #set the intial conditions
    x = np.full(1, a)
    y = np.full((1, nv), y_a)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        #calculate y_i + 1
        yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi + h_step > b):
            #take a smaller step 
            h = b - xi
            
            #recalculate y_i + 1
            yi_new, h_new, h_step = rk4_mv_ad(dydx, xi, yi, nv, h, tol)
            
            #break
            flag = 0
            
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x, xi)
        y_new = np.zeros((len(x), nv))
        y_new[0:len(x) - 1, :] = y
        y_new[-1, :] = yi[:]
        del y
        y = y_new
        
        #prevent too many iterations
        if(i >= imax):
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
        
        #iterate
        i += 1
        
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i, xi, h_step, b)
        print(s)
        
        #break if new xi is == b
        if(xi == b):
            flag = 0
        
    return x, y

### Perform the integration

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#perform the integration
x, y = rk4_mv(dydx, a, b, y_0, tolerance)

### Plot the result

In [ ]:
plt.figure(figsize = (8,7))
plt.plot(x, y[:,0], 'o', label = 'y(x)')
plt.plot(x, y[:,1], 'o', label = 'dydx(x)')
xx = np.linspace(0, 2.0*np.pi, 1000)
plt.plot(xx, np.sin(xx), label = 'sin(x)')
plt.plot(xx, np.cos(xx), label = 'cos(x)')
plt.axhline(0, color = 'black', lw = 0.8, ls = '--')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(frameon = False)

### Plot the error


In [ ]:
plt.figure(figsize = (8,7))
y_error = (y[:, 0] - np.sin(x))
dydx_error = (y[:, 1] - np.cos(x))
plt.plot(x, y_error, label = "y(x) Error")
plt.plot(x, dydx_error, label = "dydx(x) Error")
plt.axhline(0, color = 'black', lw = 0.8, ls = '--')
plt.legend(frameon = False)